# [2021 Week 48 | Power BI: Create a Waffle Chart](http://www.workout-wednesday.com/pbi-2021-w48/)

>**Table of contents:**
>
>&ensp;&ensp;[Introduction](#Introduction) <br>
>&ensp;&ensp;[Instructions](#Instructions) <br>
>&ensp;&ensp;[Workings](#Workings) <br>
>&ensp;&ensp;[Results](#Results)


### Introduction
This challenge is to visualize the Superstore Sales Data using waffle chart. 
    
The data is available [here](https://data.world/rodyzakovich/workout-wednesday-2018-week-2).

Figure 1 and Figure 2 show the solution provided in the [challenge's page](http://www.workout-wednesday.com/pbi-2021-w48/).

<center><img src='https://drive.google.com/uc?export=view&id=18Xd1zIkaRz_Mhc_Rb3FNYc6tYGxQW3iL'></center>
<p style="text-align: center;">Figure 1</p><br>

<center><img src='https://drive.google.com/uc?export=view&id=16WoKjpRvUMW4RZBqhymmRpU2Nsq-8Mam'></center>
<p style="text-align: center;">Figure 2</p>

### Instructions
- Calculate the % of Grand Total Sales that each sector contributes.
- Create a waffle chart for each sector.
- Add a chiclet slicer for the user to select the desired item(s).
- Add a text box/smart narrative based on the selected item(s).

### Workings

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('https://query.data.world/s/ppv2bq6nvnfdsrlrrcgwfcxnrwj67g', 
                   usecols=['Segment', 'Sub-Category', 'Sales'])

segments = df['Segment'].unique().tolist()

print('First 5 rows the dataset: \n', df.head(), '\n')
print('Unique entries in the Segment column: \n', segments)

First 5 rows the dataset: 
      Segment Sub-Category     Sales
0   Consumer    Bookcases  261.9600
1   Consumer       Chairs  731.9400
2  Corporate       Labels   14.6200
3   Consumer       Tables  957.5775
4   Consumer      Storage   22.3680 

Unique entries in the Segment column: 
 ['Consumer', 'Corporate', 'Home Office']


We only have 3 segments in the dataset, namely Consumer, Corporate and Home Office.

The next part defines a function to compute the percent contribution and the total sales of each segment.

In [3]:
def consider(lst):
    total, sales = 0, 0
    output = {}
    
    if lst == []:
        total += df['Sales'].sum()
        for segment in segments:
            sales += df[df['Segment']==segment]['Sales'].sum()
            output[segment] = [round(sales / total * 100), round(sales)]
            sales = 0
    
    else:
        for item in lst:
            total += df[(df['Sub-Category']==item)]['Sales'].sum()
        for segment in segments:
            for item in lst:
                sales += df[(df['Sub-Category']==item) & (df['Segment']==segment)]['Sales'].sum()
                output[segment] = [round(sales / total * 100), round(sales)]
            sales = 0
    
    return output

The output of the function defined above is a dictionary, with the three segments as the keys and each takes a list as their value. <br>
For each list, the first element is the percent contribution and followed by the total sales of each segment. <br>
The cell below shows the function's output, taking an empty list as its input.

In [4]:
sales = consider([])
sales

{'Consumer': [51, 1161401],
 'Corporate': [31, 706146],
 'Home Office': [19, 429653]}

We then proceed to define the function to make the waffle chart.

In [5]:
import plotly.graph_objects as go
import numpy as np

In [6]:
def fill_array(n):
    # take an int as input
    # return an array filled with 
    # (100-n) 0s and n 1s
    
    arr = np.zeros((10, 10))
    
    remainder = n % 10
    row = int(n/10)
    
    if remainder != 0:
        arr[:row,:] = 1
        arr[row, :remainder] = 1
    else:
        arr[:row:,:] = 1
    
    return arr

# colorscale
colour = [[0, '#AFEEEE'], # lighter (Pale Turquoise)
          [1, '#2F4F4F']] # darker (Dark Slate Gray)

def waffle(segment, lst):
    # segment is either 'Consumer', 'Corporate' or 'Home Office'
    # lst is the list of items selected, empty list means none is selected
    # return a waffle chart (made from heatmap)
    
    percentages = consider(lst)
    percentage = percentages[segment][0]
    
    fig = go.Figure(go.Heatmap(z=fill_array(percentage),
                               xgap=3, ygap=3,
                               colorscale=colour, showscale=False, 
                               hoverinfo='skip'))

    fig.update_layout(width=460, height=460, 
                      plot_bgcolor='white',
                      xaxis = {'title': '<b>{}%</b>'.format(percentage), 
                               'title_font_size': 30,
                               'color': '#2F4F4F',
                               'showticklabels': False}, 
                      yaxis = {'visible': False}, 
                      title = segment,
                      title_x = 0.5,
                      title_y = 0.01,
                      title_font_color = '#2F4F4F',
                      title_font_size = 30)


    return fig

Lastly, we create the dashboard.

In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

In [8]:
# button style (colour)
active_button = {'height':'31px', 
                 'color':'#2F4F4F',
                 'background-color':'#ADD8E6',
                 'border-color':'white',
                 'margin-bottom':'3px'}

normal_button = {'height':'31px', 
                 'color':'#2F4F4F',
                 'background-color':'#EEF7FA',
                 'border-color':'white',
                 'margin-bottom':'3px'}

# default text
non_selected = html.H4([
    'Of all superstore sales made, ',
    html.B('{}%'.format(sales['Consumer'][0])),
    ' were from the consumer sector with a total of ${:,} in sales. '.format(sales['Consumer'][1]),
    html.B('{}%'.format(sales['Corporate'][0])),
    ' came from the corporate sector, representing ${:,}. The remaining '.format(sales['Corporate'][1]),
    html.B('{}%'.format(sales['Home Office'][0])),
    ' of sales came from the home office sector, representing ${:,}.'.format(sales['Home Office'][1])])

In [9]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = dbc.Container(
    [
        # header
        dbc.Row(dbc.Col(html.H4(html.B('Superstore Sales Data by Sector'),
                                        style={'margin-top':'6px', 'margin-left':'15px'}),
                        style={'background-color':'#EEF7FA', 'height':'6vh'})
               ),
    
        # contents
        dbc.Row(
            [
                # left panel
                dbc.Col(
                    [
                        html.H5('Select an Item', style={'color':'#2F4F4F', 
                                                         'margin-bottom':'0px'}),
                        
                        html.Hr(style={'margin-top':'0px', 'margin-bottom':'0px'}),
                        
                        dbc.Card(
                            dbc.ButtonGroup(
                                [
                                    dbc.Button(item, id=item, n_clicks=0,
                                               style=normal_button)
                                    for item in sorted(df['Sub-Category'].unique())
                                ],
                                vertical=True
                            ),style={'border-color':'white'})
                    ], 
                    width=2, 
                    style={'margin-top':'10px', 
                           'margin-bottom':'10px', 
                           'margin-left':'15px'}),
                
                # graphs & text
                dbc.Col(
                    [
                        # waffle x 3
                        dbc.Row(
                            [
                                dcc.Graph(figure=waffle(segment,[]), id=segment,
                                          style={'margin-top':'-65px', 
                                                 'margin-left':'-30px',
                                                 'margin-right':'-50px',
                                                 'z-index':'-1'})
                                for segment in ['Consumer', 'Corporate', 'Home Office']
                            ],
                            style={'height':'500px'}),
                        
                        # text
                        dbc.Row(non_selected, id='text',
                                style={'margin-top':'-35px','margin-left':'5px', 'margin-right': '100px'})
                    ])
            ]
        ),
        
        # footer
        dbc.Row(dbc.Col(html.H4(html.B('WoW2021 | Power BI | Week 48'),
                                        style={'margin-top':'6px', 'margin-left':'15px'}),
                        style={'background-color':'#EEF7FA', 'height':'6vh'})),
    ],
    
    fluid=True
)

selection = []

for item in sorted(df['Sub-Category'].unique()):
    @app.callback(
        Output(item, 'active'),
        Output(item, 'style'),
        Input(item, 'n_clicks'),
        State(item, 'id')
    )
    def activeButton(click, item):
        if click != 0:
            if click % 2 == 1:
                selection.append(item)
                return True, active_button
            else:
                selection.remove(item)
                return False, normal_button
        return dash.no_update
    
@app.callback(
    [Output(segment, 'figure') for segment in ['Consumer', 'Corporate', 'Home Office']],
    Output('text', 'children'),
    [Input(item, 'active') for item in sorted(df['Sub-Category'].unique())],
)
def updateGraph(*items):
    for item in items:
        if item:
            sales = consider(selection)
            narrative = html.H4([
                'Of {} Superstore sales made, '.format(', '.join(selection)),
                html.B('{}%'.format(sales['Consumer'][0])),
                ' were from the consumer sector with a total of ${:,} in sales. '.format(sales['Consumer'][1]),
                html.B('{}%'.format(sales['Corporate'][0])),
                ' came from the corporate sector, representing ${:,}. The remaining '.format(sales['Corporate'][1]),
                html.B('{}%'.format(sales['Home Office'][0])),
                ' of sales came from the home office sector, representing ${:,}.'.format(sales['Home Office'][1])])
            return waffle('Consumer', selection), waffle('Corporate', selection), waffle('Home Office', selection), narrative
    
    return waffle('Consumer', selection), waffle('Corporate', selection), waffle('Home Office', selection), non_selected



if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
#     app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### Results

Figure 3 and Figure 4 show the final solution.<br>
<br><center><img src='https://drive.google.com/uc?export=view&id=1bdrFZCfgqEbWEKLm6WNFc2nO74HNh3Id'></center>
<p style="text-align: center;">Figure 3</p><br>
<center><img src='https://drive.google.com/uc?export=view&id=1K4Y0JKtaiZH9PKUEZYVDRWkuosMAZoeJ'></center>
<p style="text-align: center;">Figure 4</p>